# *`LIBRETA 2: preparación de los datos para el entrenamiento`*
---

El objetivo de esta libreta es generar un archivo apto para que el modelo sea entrenado.
Dicho de manera poco formal, se usará el resultado de la *libreta 1* para generar las entradas de la *libreta 3* 

Se pretende que la libreta nos ayude a generar intuiciones en como explorar los datos y como procesar los datos.

Durante toda la libreta se usará **Matplotlib** en su *filosofía orientada a objetos*.

In [19]:
%matplotlib inline
# Linea para ignorar los warnins, en su mayoria son respecto a operaciones con arreglos vacios.
import warnings
warnings.filterwarnings('ignore')


from collections import defaultdict
from collections import Counter
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from nltk import FreqDist
from nltk.corpus import stopwords
from nltk import ngrams
from nltk.lm.preprocessing import flatten

import re


## **Exploratory Data Analisys(EDA)**
---
Usualmente, hay que transformar la información para que sea más entendible por la máquina, A esto se le llama **preprocesamiento**. El EDA siempre inicia con un vistazo rápido a lo datos. Evidentemente lo primero es poder guardar el corpus en alguna variable de Python. 

Para esta tarea se ha creado una función que lee la información para del archivo 'conferencias_matutinas_matutinas_amlo(2018)', el cual es una muestra del texto en 'conferencias_matutinas_matutinas_amlo'. 

Con la filealidad de dejar el corpus lo más *'crudo'* posible se ha tomado cada **renglón** como una **sentencia** con un solo **token**, la sentencia en sí. Esto no es óptimo pues una sentencia debe de ser un listado de tokens que en su conjunto forman una sola *idea*, lo cual no está garantizado en el formato del *corpus crudo*.

In [113]:
def Leer_corpus(path,nrows=-1):
    corpus_raw	= pd.DataFrame(columns=['sentencias'])
    if nrows > 0 :
        with open(path,'r',encoding="utf-8") as frame:
            contador = 0;
            for linea in frame:
                if contador == nrows :
                    break
                auxiliar = pd.DataFrame( [linea],columns=['sentencias'])
                corpus_raw = pd.concat( [corpus_raw, auxiliar ] ,ignore_index=True)
                contador=contador+1					
    else:
        with open(path,'r',encoding="utf-8") as frame:
            for linea in frame:
                auxiliar = pd.DataFrame( [linea],columns=['sentencias'])
                corpus_raw = pd.concat( [corpus_raw, auxiliar ] ,ignore_index=True)
    return corpus_raw

path ="./conferencias_matutinas_amlo.txt"
corpus_crudo = Leer_corpus(path,8000)

corpus_crudo.head()

,sentencias
0,Buenas tardes.\n
1,"Sí. Es que son buenos días, pero llegué tarde,..."
2,Y hoy nos acompañó la jefa de gobierno de la C...
3,"Lo mismo del programa de protección, de apoyo,..."
4,"Mañana, les adelanto, se van a dar a conocer e..."


### Preprosesamiento 
---



In [119]:
#Procesamiento que modifica todo lo que se nos pudo ocurrir. Se busca cualquier excusa minimamente 
# coherente para eliminar 
def todo_UNK(sentencia):
        for palabra in sentencia:
            if(palabra != '<UNK>'):
                return False
        return True

def Procesamiento_invasivo (corpus):

    # <- Una lista de listas(sublistas). Cada sublistas tiene solo un elemento.
    corpus_preprocesado = corpus_crudo.values.tolist()     

    #Se eliminan las sentencias que tengan numero. 
    corpus_preprocesado = [ sentencia if (re.search(r'\d+',sentencia[0]) == None) else []   
                           for sentencia in corpus_preprocesado]
    corpus_preprocesado = [ sentencia for sentencia in corpus_preprocesado if sentencia != []]


    #Se transforma a lista de listas y se aplica el lower
    # NOTA: el lower tambien funiona para palabras con acentos 
    corpus_preprocesado = [ re.sub(r'\W',' ',sentencia[0]).lower()  
                           for sentencia in corpus_preprocesado]


    #Se quitan los acentos
    vocales = (
        ('á' , 'a'),
        ('é', 'e'),
        ('í' , 'i'),
        ('ó' , 'o'),
        ('ú' , 'u')
    )
    for vocales_acentuadas,vocales_regulares in vocales:
        corpus_preprocesado = [ sentencia.replace(vocales_acentuadas,vocales_regulares) 
                           for sentencia in corpus_preprocesado]
        

    #Se separan las letras 
    corpus_preprocesado = [ sentencia.split()  
                           for sentencia in corpus_preprocesado]
    corpus_preprocesado = [ sentencia for sentencia in corpus_preprocesado if sentencia != []]



    #Se Omiten las sentencias que tengan 2 o menos palabras
    #   Se aprovecha para remover tambien las sentencias que en primera intancia no tenian nada
    corpus_preprocesado = [ (sentencia if len(sentencia) >= 3 else [])   
                           for sentencia in corpus_preprocesado]
    corpus_preprocesado = [ sentencia for sentencia in corpus_preprocesado if sentencia != []]
    
    
    

    #A las sentencias muy largas se les quita algunas palabras del inicio y otras del fileal
    # Puede que este procesamiento no tenga mucho sentido. es mas un ejercicio de regex 
    # que algo ultil
    corpus_preprocesado = [ (sentencia if len(sentencia) <= 35 else sentencia[5:-6])   
                           for sentencia in corpus_preprocesado]
    
    
    #Para los siguientes preprocesamientos, se calcula la frecuencia de cada palabra
    palabras_frecuentes = Counter(flatten(corpus_preprocesado))

        
    #Se cambian las palabras poco frecuentes en el corpus
    #   Para quitar los n elementos menos frecuentes es frecuencias.most_frecuens()[ : -1 - n , -1 ]
    for index in range(len(corpus_preprocesado)):
        corpus_preprocesado[index] = [ palabra if 
                        palabras_frecuentes[palabra] > 3
                        else '<UNK>'  
                        for palabra in corpus_preprocesado[index]]
    
    #Si una sentencia tiene puras palabras UNK, se elimina
    corpus_preprocesado = [  sentencia if not todo_UNK(sentencia) else []  for sentencia in corpus_preprocesado]    
    corpus_preprocesado = [ sentencia for sentencia in corpus_preprocesado if sentencia != []]
    
    return corpus_preprocesado
#file Procesamiento intrusivo --------------------------------------------------------------------------------------

#Preprosesamiento Preprosesamiento pensado para hacer lo que pienso personalmente mas correcto
def Procesamiento_prudente (corpus):

    # <- Una lista de listas(sublistas). Cada sublistas tiene solo un elemento.
    corpus_preprocesado = corpus_crudo.values.tolist()     



    #Se transforma a lista de listas y se aplica el lower
    # NOTA: el lower tambien funiona para palabras con acentos 
    corpus_preprocesado = [ re.sub(r'\W',' ',sentencia[0]).lower()  
                           for sentencia in corpus_preprocesado]

    #Cambiamos los numeros por un token
    corpus_preprocesado = [ re.sub(r'\d+','<DGT>',sentencia)  
                           for sentencia in corpus_preprocesado]
    

    #Se quitan los acentos
    vocales = (
        ('á' , 'a'),
        ('é', 'e'),
        ('í' , 'i'),
        ('ó' , 'o'),
        ('ú' , 'u')
    )
    for vocales_acentuadas,vocales_regulares in vocales:
        corpus_preprocesado = [ sentencia.replace(vocales_acentuadas,vocales_regulares) 
                           for sentencia in corpus_preprocesado]
        

    #Se separan las letras 
    corpus_preprocesado = [ sentencia.split()  
                           for sentencia in corpus_preprocesado]
    corpus_preprocesado = [ sentencia for sentencia in corpus_preprocesado if sentencia != []]



    #Se Omiten las sentencias que tengan 2 o menos palabras
    #   Se aprovecha para remover tambien las sentencias que en primera intancia no tenian nada
    corpus_preprocesado = [ (sentencia if len(sentencia) >= 1 else [])   
                           for sentencia in corpus_preprocesado]
    corpus_preprocesado = [ sentencia for sentencia in corpus_preprocesado if sentencia != []]
    
    
    
    #Para los siguientes preprocesamientos, se calcula la frecuencia de cada palabra
    palabras_frecuentes = Counter(flatten(corpus_preprocesado))
    #Se cambian las palabras poco frecuentes en el corpus
    #   Para quitar los n elementos menos frecuentes es frecuencias.most_frecuens()[ : -1 - n , -1 ]
    for index in range(len(corpus_preprocesado)):
        corpus_preprocesado[index] = [ palabra if 
                        palabras_frecuentes[palabra] > 2
                        else '<UNK>'  
                        for palabra in corpus_preprocesado[index]]
    
    #Si una sentencia tiene puras palabras UNK, se elimina
    corpus_preprocesado = [  sentencia if not todo_UNK(sentencia) else []  for sentencia in corpus_preprocesado]    
    corpus_preprocesado = [ sentencia for sentencia in corpus_preprocesado if sentencia != []]
    
    return corpus_preprocesado
#file preprosesamiento prudente ---------------------------------------------------------------
    

#Procesamiento destinado a hacer el minimo procesamiento
def Procesamiento_simple(corpus):

    # <- Una lista de listas(sublistas). Cada sublistas tiene solo un elemento.
    corpus_preprocesado = corpus_crudo.values.tolist()     


    #Se transforma a lista de listas y se aplica el lower
    # NOTA: el lower tambien funiona para palabras con acentos 
    corpus_preprocesado = [ re.sub(r'\W',' ',sentencia[0])  
                           for sentencia in corpus_preprocesado]
    
    
    #Se separan las letras y se eliminan los sentencas vacias
    corpus_preprocesado = [ sentencia.split()  
                           for sentencia in corpus_preprocesado]
    corpus_preprocesado = [ sentencia for sentencia in corpus_preprocesado if sentencia != []]    

    
    #Se cambian las palabras poco frecuentes en el corpus por el token <UNK>
    palabras_frecuentes = Counter(flatten(corpus_preprocesado))
    for index in range(len(corpus_preprocesado)):
        corpus_preprocesado[index] = [ palabra if 
                        palabras_frecuentes[palabra] > 1
                        else '<UNK>'  
                        for palabra in corpus_preprocesado[index]]

    return corpus_preprocesado
# file preprosesamiento simple ------------------------------------------------------------

corpus_procesamiento_prudente= Procesamiento_prudente(corpus_crudo)
corpus_procesamiento_simple = Procesamiento_simple(corpus_crudo)
corpus_procesamiento_invasivo = Procesamiento_invasivo(corpus_crudo)

print((corpus_procesamiento_invasivo[:30]))
print((corpus_procesamiento_simple[:30]))
print((corpus_procesamiento_prudente[:30]))




[['si', 'es', 'que', 'son', 'buenos', 'dias', 'pero', 'llegue', 'tarde', 'porque', 'estabamos', 'tratando', 'temas', 'importantes', 'desde', 'luego', 'el', 'tema', 'de', 'la', 'seguridad'], ['proteccion', 'de', 'apoyo', 'de', 'buen', 'trato', 'de', 'evitar', '<UNK>', '<UNK>', 'a', 'nuestros', 'paisanos', 'que', 'en', 'este', 'mes', 'empiezan', 'a', 'llegar', 'a', 'los', 'pueblos', '<UNK>', 'de', 'estados', 'unidos', 'y', 'que', 'deben', 'de', 'ser'], ['a', 'dar', 'a', 'conocer', 'estos', 'programas', 'que', 'son', 'importantes', 'ya', 'en', 'esta', 'temporada', 'va', 'a', 'participar', 'la', 'policia', 'federal', 'va', 'a', 'participar', 'la', 'secretaria', 'de', 'gobernacion', 'a', 'traves', 'de', 'migracion', 'y', 'aduanas', 'porque', 'tenemos', 'que', 'evitar', 'la', 'extorsion'], ['pero', 'eso', 'es', 'un', 'adelanto', 'mañana', 'vamos', 'a', '<UNK>', 'sobre', 'las', 'acciones'], ['estoy', 'a', 'las', 'ordenes', 'de', 'ustedes'], ['nosotros', 'estamos', 'terminando', 'de', 'integra

### Objetivo: EDA
--- 
EDA nos permite visualizar la información para hacer nuestros propios descubrimientos (*insights* en inglés). La probabilidad de hacer descubrimientos es proporcional a la capacidad que se tenga para visualizar la información. 

Nuestro objetivo es preprocesar las sentencias de diferentes maneras y aplicar un EDA sobre cada uno de estos **corpus procesados**. Al fileal escogeremos la que más creamos pertinente y lo llamaremos **corpus ordenado**. El *corpus procesado* se asume que cada  elemento es una *sentencia*, a su vez cada sentencia es un conjunto de *tokens*.

La filealidad del objeto *EDA_* es ser una recopilación de herramientas para al fileal poder tener distintas métricas con las cuales comparar los derivados que se llega a hacer del corpus crudo.

---
*EDA_PLN* Contiene herramientas para hacer 'exploratory data analisys'

*   dataframe: El corpus sobre el cual se efectuara el EDA

In [33]:
class EDA_PLN():
	
	#El objeto obtiene el corpus en el formato lista de listas [ ['token' , 'token' ... 'token'] , ['token',token ...] , ...]
	def __init__(self,corpus) -> None:
		self.corpus	= corpus
		self.corpus_unidimensional = []
		for sentencia in self.corpus:
			for token in sentencia:
				self.corpus_unidimensional.append(token)

	#Toma el corpus y regresa la frecuencia de los n_grama
	#	Solo agrupa las oraciones, no añade tokens de fin e inicio de sentencia
	def get_top_ngram(self, n_grams=1):
		gramas = (ngrams(self.corpus_unidimensional,n_grams))
		frecuencias = FreqDist( gramas  )
		return frecuencias.most_common(10)


	#Grafica un histograma con la cantidad de palabras en cada sentencia
	def Palabras_por_sentencia(self,
			    eje = None,
			    tamano_titulo = 18,
			    tamano_ejes = 10,
			    titulo="Histograma de longitud por sentencias"):
		if eje is None:	
			presentacion ,eje = plt.subplots(1)


		eje.hist( list(map( lambda x : len(x)  , self.corpus )) )
		
		eje.set_title(titulo, fontsize = tamano_titulo)
		eje.set_ylabel("Frecuencia", fontsize = tamano_ejes)
		eje.set_xlabel("Longitud",fontsize = tamano_ejes)	

	#No funciona
	def Longitud_promedio_de_palabras(self,
			    eje = None):
		pass

	#No funciona
	def Palabras_mas_comunes(self,
			  	stopwords = {''},
			    eje = None):
		pass

	#No funciona
	def Palabras_menos_comunes(self,
			  	stopwords,
			    eje = None):
		pass


> 

In [130]:
EDA_tokenizado_por_palabra = EDA_PLN(corpus_crudo['sentencias'].str.split())
EDA_corpus_crudo = EDA_PLN(corpus_crudo.values.tolist())

stop=set(stopwords.words('spanish'))

with plt.style.context('bmh'):
    EDA_tokenizado_por_palabra.Caracteres_por_sentencia()


NameError: name 'EDA_PLN' is not defined

## **Preludio al modelo**
---
Lo ultimo que se hara en la libreta es guardar los corpus y generar los conjuntos de prueba y entrenamiento. La funcion **split_file** es recuperada de: https://github.com/soutsios/n-gram-language-models/blob/master/lang_mod.ipynb

In [129]:
import random
def split_file(file, out1, out2, percentage=0.75, isShuffle=True, seed=764):
    """Splits a file in 2 given the `percentage` to go in the large file."""
    random.seed(seed)
    with open(file, 'r',encoding="utf-8") as file, \
         open(out1, 'w', encoding="utf-8") as foutBig, \
         open(out2, 'w', encoding="utf-8") as foutSmall:
                nLines = sum(1 for line in file)
                file.seek(0)
                nTrain = int(nLines*percentage) 
                nValid = nLines - nTrain
                i = 0
                for line in file:
                    r = random.random() if isShuffle else 0 # so that always evaluated to true when not isShuffle
                    if (i < nTrain and r < percentage) or (nLines - i > nValid):
                        foutBig.write(line)
                        i += 1
                    else:
                        foutSmall.write(line)
                    
def corpus_procesato_a_csv(corpus,nombre_csv):
    corpus_pd = pd.DataFrame({'sentencias' :corpus}  )
    corpus_pd.to_csv(nombre_csv , index = False)
    split_file(nombre_csv,nombre_csv+'_entrenamiento',nombre_csv+'_prueba')

corpus_procesato_a_csv(corpus_procesamiento_simple,"procesado_simple")
corpus_procesato_a_csv(corpus_procesamiento_prudente,"procesado_prudente")
corpus_procesato_a_csv(corpus_procesamiento_invasivo,"procesado_invasivo")


